In [ ]:
from datetime import datetime
!git clone https://github.com/AIStream-Peelout/flow-forecast.git

Cloning into 'flow-forecast'...
remote: Enumerating objects: 16610, done.
remote: Counting objects: 100% (3467/3467), done.
remote: Compressing objects: 100% (1274/1274), done.
remote: Total 16610 (delta 2446), reused 3126 (delta 2176), pack-reused 13143
Receiving objects: 100% (16610/16610), 4.46 MiB | 4.84 MiB/s, done.
Resolving deltas: 100% (11997/11997), done.


In [ ]:
import os
!pip install -r requirements.txt
!python setup.py develop
!mkdir data

     |████████████████████████████████| 22.3 MB 165 kB/s 
     |████████████████████████████████| 5.8 MB 63.6 MB/s 
     |████████████████████████████████| 20.6 MB 778 kB/s 
     |████████████████████████████████| 503 kB 48.6 MB/s 
     |████████████████████████████████| 15.3 MB 39.1 MB/s 
     |████████████████████████████████| 3.1 MB 43.8 MB/s 
     |████████████████████████████████| 90 kB 7.0 MB/s 
     |████████████████████████████████| 94 kB 2.2 MB/s 
     |████████████████████████████████| 100 kB 7.1 MB/s 
     |████████████████████████████████| 84 kB 2.3 MB/s 
     |████████████████████████████████| 121 kB 70.3 MB/s 
     |████████████████████████████████| 93 kB 1.3 MB/s 
  Attempting uninstall: pytz
    Found existing installation: pytz 2021.1
    Uninstalling pytz-2021.1:
      Successfully uninstalled pytz-2021.1
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting u

In [ ]:
import pandas as pd
df = pd.read_csv("../../input/flowdb-sample/flowdb/01010500FVE_flow.csv")
df = df.dropna(subset=["hour_updated", "cfs", "p01m"])
df.to_csv("01010500FVE_flow.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
def make_config_file(flow_file_path, gage_id, station_id, weight_path=None, pretrained=[]):
    run = wandb.init(project="github_aistream-peelout_flow-forecast")
    wandb_config = run.config
    the_wandb_c = run.config
    print(wandb_config)
    the_config4 = {"model_name": "DARNN",
        "model_type": "PyTorch",
        "model_params": {
        "n_time_series":8,
        "forecast_history":wandb_config["forecast_history"],
        "hidden_size_encoder":wandb_config["hidden_encoder"],
        "decoder_hidden_size": wandb_config["hidden_decoder"],
        "out_feats": 1,
        "dropout": wandb_config["dropout"],
        "final_act": wandb_config["final_act"],
        "gru_lstm": False},
       "dataset_params":{"class": "default",
       "num_workers":12,
       "pin_memory": True,
       "training_path": flow_file_path,
       "validation_path": flow_file_path,
       "test_path": flow_file_path,
       "batch_size":wandb_config["batch_size"],
       "forecast_history":wandb_config["forecast_history"] - 1,
       "forecast_length":wandb_config["forecast_length"],
       "scaler": "StandardScaler",
       "train_start":1000,
       "train_end": 50000,
       "valid_start":50001,
       "valid_end": 57000,
       "sort_column": "hour_updated",
       "test_start": 57000,
       "test_end":58000,
       "target_col": ["cfs"],
       "relevant_cols": ["cfs", "tmpf", "p01m", "dwpf"], 
       "interpolate":{
           "method":"back_forward_generic",
           "params":{
               "relevant_columns":["cfs", "tmpf", "p01m", "dwpf"]}
           },
      "feature_param":
          {
            "datetime_params":{
            "hour":"cyclical",
            "month": "cyclical"
            }
          }
       },
    "training_params":
    {
       "criterion":"MSE",
       "optimizer": "SGD",
       #"criterion_params":{"baseline_method":"mean"},
    "optim_params":{
       "lr": the_wandb_c["lr"]
    },
       "epochs": 12,
       "batch_size":wandb_config["batch_size"]
    },
    "early_stopping":{
        "patience":3
    },
    "GCS": False,
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["MSE", "MAPE"],
   "inference_params":
   {     
         "datetime_start":"2020-05-31",
          "hours_to_forecast":336, 
          "num_prediction_samples": 20,
          "test_csv_path":flow_file_path,
          "decoder_params":{
            "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1},
          "dataset_params":{
             "file_path": flow_file_path,
             "sort_column": "hour_updated",
             "scaling": "StandardScaler",
             "forecast_history": wandb_config["forecast_history"] - 1,
             "forecast_length":wandb_config["forecast_length"],
             "relevant_cols": ["cfs", "tmpf", "p01m", "dwpf"],
             "target_col": ["cfs"],
             "interpolate_param":{
                 "method":"back_forward_generic",
                 "params":{"relevant_columns":["cfs", "tmpf", "p01m", "dwpf"]}
                 }, 
            "feature_params":
        {
         "datetime_params":{
            "hour":"cyclical",
            "month": "cyclical"
         
     }
             }
          }
          } 
    }

      
    if weight_path:
        the_config4["weight_path"] = weight_path
    wandb.config.update(the_config4)
    print("config made")
    return the_config4
  
wandb_sweep_config_full = {
  "name": "Default sweep",
  "method": "grid",
  "parameters": {
        "forecast_length":{
            "values":[1]},
          
        "final_act": {
          "values":["ReLU", None]},

        "batch_size": {
            "values": [200, 400, 600]
        },
        "lr":{
            "values":[0.001, 0.0001, .01]
        },
        "forecast_history":{
            "values":[14, 20, 24, 48]
        },
        "out_seq_length":{
            "values":[1]
        },
        "hidden_encoder":
        {
            "values":[32, 64, 128, 256]
        },
        "hidden_decoder":
        {
            # 32, 64, 128 
            "values":[32, 64, 128, 256]
        }, 
        "dropout":
        {
            "values": [0.1, 0.3, 0.5, 0.8]
        }
  
    }
}

In [ ]:
import wandb
from flood_forecast.trainer import train_function 
sweep_id = wandb.sweep(wandb_sweep_config_full, project="github_aistream-peelout_flow-forecast")
#sweep_id = "03l75ilz"
file_path = "01010500FVE_flow.csv"
wandb.agent(sweep_id, lambda: train_function("PyTorch", make_config_file(file_path, "01010500", "FVE", None)))

Create sweep with ID: l2hojbnt
Sweep URL: https://wandb.ai/igodfried/github_aistream-peelout_flow-forecast/sweeps/l2hojbnt


wandb: Agent Starting Run: 24v6ddkb with config:
wandb: 	batch_size: 200
wandb: 	dropout: 0.1
wandb: 	final_act: ReLU
wandb: 	forecast_history: 14
wandb: 	forecast_length: 1
wandb: 	hidden_decoder: 32
wandb: 	hidden_encoder: 32
wandb: 	lr: 0.001
wandb: 	out_seq_length: 1
wandb: Currently logged in as: igodfried (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


{'batch_size': 200, 'dropout': 0.1, 'final_act': 'ReLU', 'forecast_history': 14, 'forecast_length': 1, 'hidden_decoder': 32, 'hidden_encoder': 32, 'lr': 0.001, 'out_seq_length': 1}
config made



CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



Run 24v6ddkb errored: TypeError("forward() got an unexpected keyword argument 'dim'")
wandb: ERROR Run 24v6ddkb errored: TypeError("forward() got an unexpected keyword argument 'dim'")
wandb: Agent Starting Run: 13ecsoul with config:
wandb: 	batch_size: 200
wandb: 	dropout: 0.1
wandb: 	final_act: ReLU
wandb: 	forecast_history: 14
wandb: 	forecast_length: 1
wandb: 	hidden_decoder: 32
wandb: 	hidden_encoder: 32
wandb: 	lr: 0.0001
wandb: 	out_seq_length: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep

CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



{'batch_size': 200, 'dropout': 0.1, 'final_act': 'ReLU', 'forecast_history': 14, 'forecast_length': 1, 'hidden_decoder': 32, 'hidden_encoder': 32, 'lr': 0.0001, 'out_seq_length': 1}
config made


Run 13ecsoul errored: TypeError("forward() got an unexpected keyword argument 'dim'")
wandb: ERROR Run 13ecsoul errored: TypeError("forward() got an unexpected keyword argument 'dim'")
wandb: Agent Starting Run: 5g7tljh6 with config:
wandb: 	batch_size: 200
wandb: 	dropout: 0.1
wandb: 	final_act: ReLU
wandb: 	forecast_history: 14
wandb: 	forecast_length: 1
wandb: 	hidden_decoder: 32
wandb: 	hidden_encoder: 32
wandb: 	lr: 0.01
wandb: 	out_seq_length: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep

CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



{'batch_size': 200, 'dropout': 0.1, 'final_act': 'ReLU', 'forecast_history': 14, 'forecast_length': 1, 'hidden_decoder': 32, 'hidden_encoder': 32, 'lr': 0.01, 'out_seq_length': 1}
config made


Run 5g7tljh6 errored: TypeError("forward() got an unexpected keyword argument 'dim'")
wandb: ERROR Run 5g7tljh6 errored: TypeError("forward() got an unexpected keyword argument 'dim'")
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


In [ ]:
!wget -O 

wget: option requires an argument -- 'O'
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.
